In [1]:
!pip install groq

In [2]:
from google.colab import userdata

GROQ_API_KEY = userdata.get('groq_api_key')
PPLX_API_KEY = userdata.get('pplx_api_key')

ModuleNotFoundError: No module named 'google'

In [ ]:
from groq import Groq

client = Groq(api_key=GROQ_API_KEY)

In [ ]:
SYS_PROMPT = """\
Your task is to split a post into a list of claims.

ONLY respond with the claims, and nothing else. Do NOT number the claims.

Respond with a separate claim on a new line.

## Example Input:
Author: SWAFF
Content: We could cure cancer, but we're feedin' our money to defense contractors and foreign gov'ts. and LGBTQ BS #CancerVaccine is our battle cry, showin' how we neglect our health for war. We need to make #MAHA lifestyle the American lifestyle.

## Example Output:
We could cure cancer.
Money is being sent to foreign governments and defense contractors instead of cancer research.
The focus on LGBTQ issues distracts from cancer research and the need for a cancer vaccine.
The US neglects its health in favor of funding war efforts.
The MAHA lifestyle should be adopted as the American lifestyle.

## Example Input:
Author: Charlie Kirk
Content: The speed, depth, and power of DOGE has shocked both supporters and critics. I’m surprised and I’m thrilled. It’s fundamentally reshaping the future of what’s possible in government. Historic.

## Example Output:
The speed, depth, and power of DOGE has shocked both supporters and critics.
"""

USER_PROMPT = """\
Author: Shadow of Ezra
Content: The lawsuit against Sean “Diddy” Combs and Jay-Z over the alleged r*pe of a 13-year-old girl in 2000 has been dismissed by attorney Tony Buzbee, who was representing Jane Doe. Jane Doe dismissed the case "with prejudice," meaning it cannot be re-filed again in the future. Jay-Z a.k.a. Shawn Carter has declared victory."""

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": SYS_PROMPT
        },
        {
            "role": "user",
            "content": USER_PROMPT,
        },
    ],
    model="llama-3.2-90b-vision-preview",
    stop=None,
    temperature=0,
)

response = chat_completion.choices[0].message.content

claims = [v.strip() for v in response.split("\n")]

claims

['A lawsuit was filed against Sean "Diddy" Combs and Jay-Z over the alleged rape of a 13-year-old girl in 2000.',
 'The lawsuit against Sean "Diddy" Combs and Jay-Z has been dismissed by attorney Tony Buzbee.',
 'The lawsuit was dismissed "with prejudice," meaning it cannot be re-filed again in the future.',
 'Jay-Z has declared victory following the dismissal of the lawsuit.']

In [ ]:
import requests

def call_perplexity(messages):
  url = "https://api.perplexity.ai/chat/completions"

  payload = {
      "model": "sonar",
      "messages": messages,
      "temperature": 0,
      "return_images": False,
      "return_related_questions": False,
      "search_recency_filter": "month",
  }
  headers = {
      "Authorization": f"Bearer {PPLX_API_KEY}",
      "Content-Type": "application/json"
  }

  response = requests.request("POST", url, json=payload, headers=headers)

  return response

In [ ]:
import json

def find_sources_for_claim(post, claim):
  return json.loads(call_perplexity([
      {
          "role": "system",
          "content": """\
Your job is to analyze the veracity of a claim.

Respond using 2-3 sentences.

Only respond with evidence relating directly to the claim; only use the full text of the post as additional context.
"""
      },
      {
          "role": "user",
          "content": f"""\
Here is the full text of a post:
"{post}"


Find evidence to support or reject this claim (and ONLY this claim) from the post:
"{claim}".
"""
      }
  ]).text)

POST = """The lawsuit against Sean “Diddy” Combs and Jay-Z over the alleged r*pe of a 13-year-old girl in 2000 has been dismissed by attorney Tony Buzbee, who was representing Jane Doe. Jane Doe dismissed the case "with prejudice," meaning it cannot be re-filed again in the future. Jay-Z a.k.a. Shawn Carter has declared victory."""

claims_sources_explanations = []
for claim in claims:
  res = find_sources_for_claim(POST, claim)
  sources = res["citations"]
  explanation = res["choices"][0]["message"]["content"]

  print(explanation)
  print("---")

  claims_sources_explanations.append((claim, sources, explanation))

The claim that "A lawsuit was filed against Sean 'Diddy' Combs and Jay-Z over the alleged rape of a 13-year-old girl in 2000" is supported by the following evidence:

- A civil lawsuit was indeed filed against Sean "Diddy" Combs and Jay-Z, accusing them of raping a 13-year-old girl at a house party following the MTV Video Music Awards in 2000[1][2].
- The lawsuit was initially filed against Combs in October and later amended to include Jay-Z as a defendant in December[1].
---
The claim that "the lawsuit against Sean 'Diddy' Combs and Jay-Z has been dismissed by attorney Tony Buzbee" is not supported by the available evidence. According to the search results, the lawsuit was voluntarily dismissed by the plaintiff (Jane Doe) through her attorney, but there is no specific mention that Tony Buzbee was the one who dismissed the lawsuit on behalf of the plaintiff. Instead, Tony Buzbee is mentioned as having filed more than two dozen lawsuits against Combs, including the one that was dismisse

In [ ]:
evaluations = []

def rate_claim(post, claim, evidence):
  prompt1 = f"""\
Here is the full text of a post:
"{post}"


Find evidence to support or reject this claim from the post:
"{claim}".
"""

  prompt2 = "Given this explanation, respond with whether the claim is ACCURATE or MISLEADING.\n\nOnly respond with ACCURATE or MISLEADING, nothing else."

  completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt1,
        },
        {
            "role": "assistant",
            "content": evidence,
        },
        {
            "role": "user",
            "content": prompt2,
        },
    ],
    model="llama-3.1-8b-instant",
    stop=None,
    temperature=0,
  )

  response = completion.choices[0].message.content

  return response


for claim, source, evidence in claims_sources_explanations:
  decision = rate_claim(POST, claim, evidence)

  evaluation = {
      "claim": claim,
      "sources": sources,
      "evidence": evidence,
      "is_misleading": decision == "MISLEADING"
  }

  print(evaluation)

  evaluations.append(evaluation)


{'claim': 'A lawsuit was filed against Sean "Diddy" Combs and Jay-Z over the alleged rape of a 13-year-old girl in 2000.', 'sources': ['https://www.latimes.com/california/story/2025-02-14/accuser-drops-rape-lawsuit-against-jay-z-and-sean-diddy-combs', 'https://abc7.com/post/civil-case-accusing-sean-diddy-combs-shawn-jay-carter-sexual-assault-dismissed/15908008/', 'https://www.cbsnews.com/news/jay-z-sean-diddy-combs-sexual-assault-lawsuit-dropped-by-accuser/', 'https://abcnews.go.com/Entertainment/civil-case-accusing-jay-z-diddy-rape-dismissed/story?id=118847148'], 'evidence': 'The claim that "A lawsuit was filed against Sean \'Diddy\' Combs and Jay-Z over the alleged rape of a 13-year-old girl in 2000" is supported by the following evidence:\n\n- A civil lawsuit was indeed filed against Sean "Diddy" Combs and Jay-Z, accusing them of raping a 13-year-old girl at a house party following the MTV Video Music Awards in 2000[1][2].\n- The lawsuit was initially filed against Combs in October 